# Global sensitivity Analysis for PULPO

In this notebook we show the workflow for the global sensitivity analysis (GSA) in PULPO.

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
sys.path.append('../develop_test')
import stats as pulpostats
import numpy as np
import pandas as pd

## 1. Defining the case

The case for which the sensitivity analysis will be performed for is on a solution of the LP:

$$
    \begin{align}
        & \underset{s, slack}{\text{min}}  && z_h \\
        & \text{s.t.}   && \sum_{j}(a_{i,j}\cdot s_j) = f_i && \forall i \\
        &               && s_j^{low} \leq s_j \leq s_j^{high} && \forall j \\
        &               && z_h = \sum_e \sum_j (q_{h,e}\cdot b_{e,j} \cdot s_j) && \forall h \\
    \end{align}
$$


### 1.1. Rice husk problem


In [ ]:
import bw2data as bd
if "rice_husk_example" in bd.projects:
    bd.projects.set_current("default")
    bd.projects.delete_project("rice_husk_example", delete_dir=True)
    print("Deleted rice_husk_example project")
case_study = pulpostats.RiceHuskCase()
db = bd.Database("rice_husk_example_db")
print(len(db))
case_study.create_pulpo_worker()
case_study.define_problem()
result_data = case_study.solve_and_summarize(file_name="rice_husk_test")

### 1.2. Defining the electricity showcase problem

In [ ]:
case_study = pulpostats.ElectricityCase()
case_study.create_pulpo_worker()
case_study.define_problem()
result_data = case_study.solve_and_summarize(file_name='electricity_test')

### 1.3. Defining the Ammonia case study

In [ ]:
case_study = pulpostats.AmmoniaCase()
case_study.create_pulpo_worker()
case_study.define_problem()
result_data = case_study.solve_and_summarize(file_name='ammonia_test')


## 2. Filtering out negletable uncertain parameters

**Reformulating the problem for the sensitivity analysis**

We only consider uncertainty in the $B$ and $Q$ parameter matrizes. The scaling vector is given by the optimal solution.

We will look at the environmental impact objective:

$$
    e(Q, B) =  Q \cdot B \cdot s
$$

In [ ]:
paramfilter = pulpostats.ParameterFilter(
    result_data=result_data, 
    lci_data=case_study.pulpo_worker.lci_data, 
    choices = case_study.choices,
    demand = case_study.demand,
    method = case_study.method
    )

Using the basic scaling vector which only includes the optimal choices in the scaling vector

In [ ]:
scaling_vector_series = paramfilter.prepare_sampling(scaling_vector_strategy='naive')

Using the scaling vector constructed from all choices

In [ ]:
scaling_vector_series = paramfilter.prepare_sampling(scaling_vector_strategy='constructed_demand')

Compute the LCA scores and return the characterized inventory to be used for the filtering

In [ ]:
lca_score, characterized_inventory = paramfilter.compute_LCI_LCIA(scaling_vector_series)

Plot the largest contributors

In [ ]:
paramfilter.plot_top_processes(characterized_inventory, top_amount=9)

Filtering out the inventoryflows $B_{i,j}$ that have a neglectable impact

In [ ]:
cutoff = 0.00002
filtered_inventory_indcs = paramfilter.filter_inventoryflows(characterized_inventory, lca_score, cutoff)

In [ ]:
filtered_characterization_indcs = paramfilter.filter_characterization_factors(filtered_inventory_indcs)

## 3. Getting the uncertainty of the parameter values

### 3.1. Interventions flows

Extract the metadata containing the uncertainty information to the filtered intervention flows and seperate the metadata into the parameters with and without defined uncertainty information

In [ ]:
uncertainty_importer = pulpostats.UncertaintyImporter(lci_data=case_study.pulpo_worker.lci_data)
backgroundDB_filtInv_indcs = uncertainty_importer.get_intervention_indcs_to_db('ecoinvent-3.10-cutoff', filtered_inventory_indcs)
foregroundDB_filtInv_indcs = uncertainty_importer.get_intervention_indcs_to_db('ammonia-reduced', filtered_inventory_indcs)
bg_inventory_metadata_df = uncertainty_importer.get_intervention_meta(inventory_indices=backgroundDB_filtInv_indcs)
fg_inventory_metadata_df = uncertainty_importer.get_intervention_meta(inventory_indices=foregroundDB_filtInv_indcs)
bg_inventory_defined,  bg_inventory_undefined = uncertainty_importer.separate(bg_inventory_metadata_df)
fg_inventory_defined,  fg_inventory_undefined = uncertainty_importer.separate(fg_inventory_metadata_df)


Apply the triangular strategy using bound interpolation to the missing intervention uncertainty parameters in the background database

In [ ]:
if_bg_triangular_strategy = pulpostats.TriangularBoundInterpolationStrategy(
    metadata_df=bg_inventory_metadata_df,
    defined_uncertainty_metadata=bg_inventory_defined,
    undefined_uncertainty_indices= bg_inventory_undefined,
    noise_interval={'min':.1, 'max':.1}
    )

Apply the Uniform strategy for the uncertainty parameters in the foreground database

In [ ]:
if_fg_uniform_strategy = pulpostats.UniformBaseStrategy(
    metadata_df=fg_inventory_metadata_df,
    defined_uncertainty_metadata=fg_inventory_defined,
    undefined_uncertainty_indices=fg_inventory_undefined,
    upper_scaling_factor = .5,
    lower_scaling_factor = .5,
    noise_interval={'min':.2, 'max':.2}
)

### 3.2. Characterization factors

Extract the metadata containing the uncertainty information to the filtered characterization factors and seperate the metadata into the parameters with and without defined uncertainty information

In [ ]:
characterization_metadata_df = uncertainty_importer.get_cf_meta(
    characterization_indices=filtered_characterization_indcs,
    method=case_study.method
    )
cf_defined,  cf_undefined = uncertainty_importer.separate(characterization_metadata_df)

Apply the triangular strategy using predefined scaling factors to the missing uncertainty parameters

In [ ]:
upper_scaling_factor_cf = 0.15
lower_scaling_factor_cf = 0.15
cf_triangular_strategy = pulpostats.TriangluarBaseStrategy(
    metadata_df=characterization_metadata_df,
    defined_uncertainty_metadata=cf_defined,
    undefined_uncertainty_indices=cf_undefined,
    upper_scaling_factor = upper_scaling_factor_cf,
    lower_scaling_factor = lower_scaling_factor_cf,
    noise_interval={'min':.1, 'max':.1}
)

In [ ]:
unc_metadata = {
    'cf': cf_triangular_strategy.metadata_assigned_df,
    'if': pd.concat([if_fg_uniform_strategy.metadata_assigned_df, if_bg_triangular_strategy.metadata_assigned_df])
}

## 4. Define the global sensitivity problem
### 4.1. Define the bound/interval of the parameters

Check if all parameters have gotten uncertainty information asigned

In [ ]:
from SALib.sample import sobol as sample_method
from SALib.analyze import sobol as SA_method
N = 2**3

gsa = pulpostats.GlobalSensitivityAnalysis(
    result_data=result_data,
    lci_data=case_study.pulpo_worker.lci_data,
    unc_metadata=unc_metadata,
    sampler=sample_method,
    analyser=SA_method,
    sample_size=N,
    method=case_study.method
)
gsa_problem, all_bounds_indx_dict = gsa.define_problem()

In [ ]:
sample_data_if, sample_data_cf = gsa.sample(gsa_problem, all_bounds_indx_dict)

In [ ]:
sample_impacts, sample_characterized_inv entories = gsa.run_model(sample_data_if, sample_data_cf)

In [ ]:
total_Si = gsa.analyze(gsa_problem, sample_impacts)

In [ ]:
total_Si_metadata = gsa.generate_Si_metadata(all_bounds_indx_dict, total_Si)
colormap_base, colormap_SA_barplot = gsa.plot_top_total_sensitivity_indices(total_Si, total_Si_metadata)

In [ ]:
data_plot = gsa.plot_total_env_impact_contribution(
    sample_characterized_inventories, 
    total_Si_metadata, 
    colormap_base=colormap_base, 
    colormap_SA_barplot=colormap_SA_barplot,
    )

## 5. CC formulation

Formulate the chance constrain optimization problem. 

In the future multiple formulations might be possible, represented by different `CCFormulation` classes

In [ ]:
cc_formulation = pulpostats.CCFormulationObjIndividualNormalL1(
    unc_metadata=unc_metadata,
    pulpo_worker=case_study.pulpo_worker,
    method=case_study.method,
    choices=case_study.choices,
    demand=case_study.demand
)

Initiate epsilon solver for the Pareto Problem

In [ ]:
epsilon_pareto_solver = pulpostats.EpsilonConstraintSolver(cc_formulation)

Solve single Pareto point

In [ ]:
CC_solution = epsilon_pareto_solver.solve_single_pareto_point(lambda_level=.5)
CC_solution

Solve for an array of lambda epsilon constraints

In [ ]:
lambda_epsilon_array = np.linspace(0.5,1, 5, endpoint=False)
CC_solutions = epsilon_pareto_solver.solve(lambda_epsilon_array)

Plot Pareto front

In [ ]:
epsilon_pareto_solver.plot_pareto_front(CC_solutions, 0.03)

Compare the Pareto points

In [ ]:
epsilon_pareto_solver.compare_subsequent_paretosolutions(CC_solutions)